## Offline dictionary attack on simplified SRP
```
S
x = SHA256(salt|password)
    v = g**x % n

C->S
I, A = g**a % n

S->C
salt, B = g**b % n, u = 128 bit random number

C
x = SHA256(salt|password)
    S = B**(a + ux) % n
    K = SHA256(S)

S
S = (A * v ** u)**b % n
    K = SHA256(S)

C->S
Send HMAC-SHA256(K, salt)
S->C
Send "OK" if HMAC-SHA256(K, salt) validates
```

Note that in this protocol, the server's "B" parameter doesn't depend on the password (it's just a Diffie Hellman public key).

Make sure the protocol works given a valid password.

Now, run the protocol as a MITM attacker: pose as the server and use arbitrary values for b, B, u, and salt.

Crack the password from A's HMAC-SHA256(K, salt).


## Parameter negotiation
Carol and Steve agree on $N$ and $g$.

In [1]:
# A large safe prime (N = 2q+1, where q is prime)
# All arithmetic is done modulo N
# (generated using "openssl dhparam -text 1024")
N = """00:c0:37:c3:75:88:b4:32:98:87:e6:1c:2d:a3:32:
       4b:1b:a4:b8:1a:63:f9:74:8f:ed:2d:8a:41:0c:2f:
       c2:1b:12:32:f0:d3:bf:a0:24:27:6c:fd:88:44:81:
       97:aa:e4:86:a6:3b:fc:a7:b8:bf:77:54:df:b3:27:
       c7:20:1f:6f:d1:7f:d7:fd:74:15:8b:d3:1c:e7:72:
       c9:f5:f8:ab:58:45:48:a9:9a:75:9b:5a:2c:05:32:
       16:2b:7b:62:18:e8:f1:42:bc:e2:c3:0d:77:84:68:
       9a:48:3e:09:5e:70:16:18:43:79:13:a8:c3:9c:3d:
       d0:d4:ca:3c:50:0b:88:5f:e3"""

N = int("".join(N.split()).replace(":", ""), 16)
g = 2 # A generator modulo N

In [2]:
I = b'carol'
password = b'foobar'

## Client registration
Carol sends salt and verifier $v$ to Steve.

Steve stores salt and $v$.

In [3]:
import os
import hashlib

In [4]:
salt = os.urandom(64)
print(len(salt), salt.hex())

64 4de4a86e229bc8b2bdbc49c232372f8be2dc02d0b292f5c74e17b8c2fcb0dd4a69bce42052891fc5bc4b8a9f626260264939c9059487d7cec36166d80823c9a6


In [5]:
sha256 = hashlib.sha256()
sha256.update(salt)
sha256.update(password)
x = sha256.digest()
x.hex()

'aef39b990a10a498c14c172d22a0805528d56f73c877ea6c873ac32a18b6a1f2'

In [6]:
x = int.from_bytes(x, byteorder="big")

In [7]:
v = pow(g, x, N)
v

130010863821338766212798769022741766476173545768192241575703211151659831295347954380957738906438577800781045419865716095315043710194775051648409561725748714695965426316454364062778547624011722368703121722600375543671304844060059220310680120233401895110866908137817323383349801506135281231186901467967308098540